# NeuraEstate Business Agent

This notebook implements an AI-powered business assistant for **NeuraEstate**, a UAE-based AI real estate technology company.

The agent:
- Answers questions about NeuraEstate using business context
- Collects customer leads and contact information
- Records customer feedback for continuous improvement


## 1. Setup and Imports


In [1]:
import os
import json
from datetime import datetime
from pathlib import Path
from dotenv import load_dotenv
import fitz  # PyMuPDF
from openai import OpenAI
import gradio as gr


## 2. Environment Configuration

Load API key from `.env` file.


In [2]:
# Load environment variables
load_dotenv()

# Get OpenAI API key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY not found in .env file")

# Initialize OpenAI client
client = OpenAI(api_key=OPENAI_API_KEY)

print("OpenAI client initialized successfully")


OpenAI client initialized successfully


## 3. Business Context Loading

Load business information from the summary text file and PDF document.


In [3]:
def load_business_context():
    """Load business context from text file and PDF."""
    context_parts = []
    
    # Read business summary text file
    summary_path = Path("me/business_summary.txt")
    if summary_path.exists():
        with open(summary_path, "r", encoding="utf-8") as f:
            context_parts.append("=== Business Summary ===\n" + f.read())
    
    # Read PDF file
    pdf_path = Path("me/about_business.pdf")
    if pdf_path.exists():
        doc = fitz.open(pdf_path)
        pdf_text = ""
        for page_num in range(len(doc)):
            page = doc[page_num]
            pdf_text += page.get_text()
        doc.close()
        context_parts.append("=== Additional Business Information ===\n" + pdf_text)
    
    return "\n\n".join(context_parts)

# Load business context
BUSINESS_CONTEXT = load_business_context()
print(f"Business context loaded ({len(BUSINESS_CONTEXT)} characters)")


Business context loaded (5783 characters)


## 4. Tool Functions

Define functions that the agent can call to record customer interest and feedback.


In [4]:
def record_customer_interest(name: str, email: str, message: str = "") -> str:
    """
    Record customer interest/lead information.
    
    Args:
        name: Customer's full name
        email: Customer's email address
        message: Optional message or notes from the customer
    
    Returns:
        Confirmation message
    """
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    lead_data = {
        "timestamp": timestamp,
        "name": name,
        "email": email,
        "message": message
    }
    
    # Log to console
    print(f"\n[LEAD CAPTURED] {timestamp}")
    print(f"Name: {name}")
    print(f"Email: {email}")
    if message:
        print(f"Message: {message}")
    
    # Write to JSON file
    leads_file = Path("customer_leads.jsonl")
    with open(leads_file, "a", encoding="utf-8") as f:
        f.write(json.dumps(lead_data, ensure_ascii=False) + "\n")
    
    return f"Thank you, {name}! I've recorded your information and our team will contact you at {email} soon."


def record_feedback(question: str) -> str:
    """
    Record customer feedback or questions that the agent couldn't answer.
    
    Args:
        question: The question or feedback from the customer
    
    Returns:
        Confirmation message
    """
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    feedback_data = {
        "timestamp": timestamp,
        "question": question
    }
    
    # Log to console
    print(f"\n[FEEDBACK RECORDED] {timestamp}")
    print(f"Question: {question}")
    
    # Write to JSON file
    feedback_file = Path("customer_feedback.jsonl")
    with open(feedback_file, "a", encoding="utf-8") as f:
        f.write(json.dumps(feedback_data, ensure_ascii=False) + "\n")
    
    return "Thank you for your feedback! I've recorded your question and our team will review it to improve our services."


## 5. System Prompt and Agent Configuration

Create the system prompt that defines NeuraEstate's agent persona and available tools.


In [5]:
# Define system prompt
SYSTEM_PROMPT = f"""You are an AI assistant representing NeuraEstate, a UAE-based AI real estate technology company.

Your role is to:
1. Provide accurate and helpful information about NeuraEstate's services, mission, and capabilities
2. Answer questions about the company professionally and in character
3. Identify when customers express interest or provide contact information, and use the record_customer_interest function
4. Record questions you cannot answer using the record_feedback function

Business Context:
{BUSINESS_CONTEXT}

Guidelines:
- Stay in character as a professional representative of NeuraEstate
- Be friendly, knowledgeable, and helpful
- When a customer expresses interest, asks about pricing, wants to schedule a demo, or provides contact information (name/email), immediately use record_customer_interest
- If you cannot answer a question based on the provided context, use record_feedback to log it
- Always maintain a professional, business-appropriate tone
- Highlight NeuraEstate's AI-powered solutions and UAE market focus"""

# Define available functions for OpenAI
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "record_customer_interest",
            "description": "Record customer interest, lead information, or contact details when a customer expresses interest, asks about pricing/demos, or provides their name and email.",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "Customer's full name"
                    },
                    "email": {
                        "type": "string",
                        "description": "Customer's email address"
                    },
                    "message": {
                        "type": "string",
                        "description": "Optional message or notes from the customer"
                    }
                },
                "required": ["name", "email"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "record_feedback",
            "description": "Record customer feedback or questions that cannot be answered with the available information.",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {
                        "type": "string",
                        "description": "The question or feedback from the customer"
                    }
                },
                "required": ["question"]
            }
        }
    }
]

print("System prompt and tools configured")


System prompt and tools configured


## 6. Agent Logic

Implement the core chatbot function that handles conversation and tool calling.


In [6]:
def chat_with_agent(message, history):
    """
    Handle a single message exchange with the NeuraEstate agent.
    
    Args:
        message: User's message
        history: Conversation history (list of [user_msg, assistant_msg] pairs)
    
    Returns:
        Agent's response
    """
    # Convert history to OpenAI format
    messages = [{"role": "system", "content": SYSTEM_PROMPT}]
    
    for user_msg, assistant_msg in history:
        messages.append({"role": "user", "content": user_msg})
        messages.append({"role": "assistant", "content": assistant_msg})
    
    # Add current user message
    messages.append({"role": "user", "content": message})
    
    # Get response from OpenAI
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            tools=TOOLS,
            tool_choice="auto"
        )
        
        assistant_message = response.choices[0].message
        
        # Handle tool calls
        if assistant_message.tool_calls:
            tool_results = []
            
            for tool_call in assistant_message.tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)
                
                # Call the appropriate function
                if function_name == "record_customer_interest":
                    result = record_customer_interest(
                        name=function_args.get("name", ""),
                        email=function_args.get("email", ""),
                        message=function_args.get("message", "")
                    )
                elif function_name == "record_feedback":
                    result = record_feedback(question=function_args.get("question", ""))
                else:
                    result = "Function not recognized"
                
                tool_results.append({
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": result
                })
            
            # Add tool results and get final response
            messages.append(assistant_message)
            messages.extend(tool_results)
            
            final_response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=messages
            )
            
            return final_response.choices[0].message.content
        else:
            return assistant_message.content
    
    except Exception as e:
        return f"I apologize, but I encountered an error: {str(e)}. Please try again."


## 7. Gradio Interface

Create a user-friendly Gradio chat interface for interacting with the NeuraEstate agent.


In [7]:
# Create Gradio ChatInterface
demo = gr.ChatInterface(
    fn=chat_with_agent,
    title="NeuraEstate AI Assistant",
    description="Welcome to NeuraEstate! I'm here to answer your questions about our AI-powered real estate solutions. Feel free to ask about our services, request information, or share your contact details if you're interested.",
    examples=[
        "What services does NeuraEstate offer?",
        "Tell me about your AI Property Matcher",
        "How can I get in touch with NeuraEstate?",
        "What makes NeuraEstate different from other real estate tech companies?"
    ],
    theme=gr.themes.Soft(),
    cache_examples=False
)

print("Gradio interface created successfully")


Gradio interface created successfully


c:\Users\shafi\anaconda3\envs\ai-agentic\Lib\site-packages\gradio\chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


## 8. Launch the Interface

Start the Gradio web interface.


In [ ]:
# Launch the interface
if __name__ == "__main__" or True:
    demo.launch(share=False, server_name="127.0.0.1", server_port=7860)


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.



[FEEDBACK RECORDED] 2025-11-02 20:43:58
Question: What is the exact office location of NeuraEstate?

[LEAD CAPTURED] 2025-11-02 20:45:54
Name: Shafik Houeidi
Email: sah89@mail.aub.edu
Message: Interested in AI Property Matcher for finding cheap apartments in Dubai.

[FEEDBACK RECORDED] 2025-11-02 20:45:54
Question: What cheap apartments do you have in Dubai?
